### 导入必要的库，简单设置

In [ ]:
import pandas as pd
pd.options.display.max_columns=99
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font='Microsoft YaHei Mono')
%matplotlib inline

### 读取要处理的数据

In [ ]:
df = pd.read_csv('淄博信息港二手房详情20160110.csv')
df.head()

### 原始数据需要处理的问题
- 部分列存储了多列信息
    比如my_column_5列中的一个数据：户型：3室 2厅 1卫 楼层：18 /22，我们希望处理成：
   
| 室 | 厅 | 卫 | 所在层 | 总楼层 |
| :-: | :-: | :-: | :-: | :-: |
| 3 | 2 | 1 | 18 | 22 |

- 大部分列包含了不需要的字符：~~朝向：~~东、~~装修：~~毛坯……
- 处理完成之后需要做格式转换，发布时间应该是日期，价格、面积、室厅卫数量应该是数值……
- 需要计算出一些辅助信息，比如说根据建造年代生成房龄。

#### 由简至繁，先处理只需要剔除冗余字符的列。

In [ ]:
new_df=pd.DataFrame(index=df.index)

In [ ]:
new_df["区县"]= df['my_column'].str.replace("二手房","")
new_df["区县"].value_counts()

*貌似只有张店人在用淄博信息港。*

In [ ]:
new_df["小区"]= df['my_column_6']
new_df["小区"].value_counts().head(),new_df["小区"].value_counts().tail()

*小区名称还需要整理，为了简化，可以舍弃名称不对的数据*

In [ ]:
new_df["类型"]= df['my_column_8'].str.replace("类型：","")
new_df["类型"].value_counts()

In [ ]:
new_df["建造年代"]= df['my_column_9'].str.replace("建造年代：","").str.replace("年","")
new_df["建造年代"].value_counts()

*2010年是盖房子高潮？*

In [ ]:
new_df["装修"]= df['my_column_11'].str.replace("装修：","")
new_df["装修"].value_counts()

In [ ]:
new_df["朝向"]= df['my_column_10'].str.replace("朝向：","")
new_df["朝向"].value_counts()

*南北和南有什么区别？东西和东有什么区别？*

In [ ]:
new_df["发布时间"]=pd.to_datetime(df['my_column_14'].str.replace("发布",""),format='%Y-%m-%d')
new_df["发布时间"].value_counts()

In [ ]:
new_df["价格"]= pd.to_numeric(df['my_column_2'].str.replace("￥","").str.replace("万元",""),errors='coerce')
new_df["价格"].value_counts() 

In [ ]:
len(new_df[new_df['价格'].isnull()])

#### 接下来处理稍微复杂些的，需要拆分的列

**面积** 
- 面积：120㎡ 车:15㎡
- 面积：130㎡ 储:17㎡
- ……

In [ ]:
s1="面积：120㎡ 车:15㎡"
s2="面积：130㎡ 储:17㎡"

def deal_str(s,k):
    d={"面积":np.nan,"车":np.nan,"储":np.nan}
    if type(s)==type(""):# pandas 中的Nan值是flaot类型的，如果出现缺失值，当成字符处理就会出错。
        s=s.replace("㎡","").replace("：",":")
        l=s.split(" ")
        for i in l:
            tmpl=i.split(":")
            d[tmpl[0]]=tmpl[1]
    return d[k]

def get_area(s):
    return deal_str(s,k="面积")
def get_car(s):
    return deal_str(s,k="车")
def get_store(s):
    return deal_str(s,k="储")
    
        
get_area(s1),get_area(s2)

In [ ]:
new_df["面积"]=df["my_column_4"].map(get_area)
new_df["车库"]=df["my_column_4"].map(get_car)
new_df["储藏室"]=df["my_column_4"].map(get_store)
new_df.head()

**户型和楼层** 
- 户型：2室 2厅 1卫 楼层：4 /5 
- ……

In [ ]:
import re

In [ ]:
s="户型：2室 2厅 1卫 楼层：4 /5"
r=r".*户型：(\d+)室\s+(\d+)厅\s+(\d+)卫\s+楼层：(\d+)\s\/(\d+)"
m=re.match(r,s)
m1.group(1),m1.group(2),m1.group(3),m1.group(4),m1.group(5)

In [ ]:
new_df["室"]=df['my_column_5'].map(lambda x: re.match(r".*(\d+)室",str(x)).group(1) if re.match(r".*户型：(\d+)室",str(x)) else np.nan)
new_df["室"].value_counts()

In [ ]:
new_df["厅"]=df['my_column_5'].map(lambda x: re.match(r".*(\d+)厅",str(x)).group(1) if re.match(r".*(\d+)厅",str(x)) else np.nan)
new_df["厅"].value_counts()

In [ ]:
new_df["卫"]=df['my_column_5'].map(lambda x: re.match(r".*(\d+)卫",str(x)).group(1) if re.match(r".*(\d+)卫",str(x)) else np.nan)
new_df["卫"].value_counts()

In [ ]:
new_df["楼层"]=df['my_column_5'].map(lambda x: re.match(r".*楼层：(\d+)",str(x)).group(1) if re.match(r".*楼层：(\d+)",str(x)) else np.nan)
new_df["楼层"].value_counts()

In [ ]:
new_df["总楼层"]=df['my_column_5'].map(lambda x: re.match(r".*楼层：\d+\s+/(\d+)",str(x)).group(1) if re.match(r".*楼层：\d+\s+/(\d+)",str(x)) else np.nan)
new_df["总楼层"].value_counts()

In [ ]:
new_df.head()

In [ ]:
new_df.info()

In [ ]:
new_df[["建造年代","面积","车库","储藏室","室","厅","卫","楼层","总楼层"]]=new_df[["建造年代","面积","车库","储藏室","室","厅","卫","楼层","总楼层"]].astype(float)

In [ ]:
new_df[["车库","储藏室","室","厅","卫"]]=new_df[["车库","储藏室","室","厅","卫"]].fillna(0)

In [ ]:
new_df.to_excel("淄博信息港二手房信息20160110.xlsx")